Run import and all functions, then follow comments 

You need to create a folder called proceed1 with the path /content/proceed1 and leave the .txt file (have to manually change to NLS_14.txt for now) in /content


In [171]:
import pandas as pd
import numpy as np
from string import digits
import os, os.path
import numpy as np

In [10]:
def countTrials(textFile):
  range1 = [i for i in range(0,7)]
  fileRead = pd.read_table(textFile, usecols = range1, sep='\t', delim_whitespace=True, dtype = {"TIMESTAMP":object, "RIGHT_EYE_X":object, "RIGHT_EYE_Y":object, "RIGHT_PUPIL_SIZE":object, "LEFT_EYE_X":object, "LEFT_EYE_Y":object, "LEFT_EYE_PUPIL":object});
  fileRead.columns = ["TIMESTAMP", "RIGHT_EYE_X", "RIGHT_EYE_Y", "RIGHT_PUPIL_SIZE", "LEFT_EYE_X", "LEFT_EYE_Y", "LEFT_EYE_PUPIL"];
  #print(fileRead.iloc[150:170,0])
  rowCount = fileRead.shape[0];
  colCount = 7;
  sample = "SAMPLE";
  end = "END";
  currRow = 0;
  trialNum = 1;
  fileDir = "/content/proceed1/";
  patientNum = 14;
  while currRow < rowCount:
    startRow = 1;
    endRow = 1;
    if sample in str(fileRead.iloc[currRow, 0]):
      #print("yes"+str(currRow));
      startRow = currRow;
      currRowTemp = currRow;
      while currRowTemp < rowCount:
        if end in str(fileRead.iloc[currRowTemp, 0]):
          #print("end found"+str(currRowTemp))
          endRow = currRowTemp;
          currRow = currRowTemp;
          break;
        else:
          currRowTemp = currRowTemp + 1;
      np.savetxt(fileDir+"NLS_"+str(patientNum)+"_"+str(trialNum)+".txt", fileRead.iloc[startRow:endRow, 0:colCount], delimiter=" ", fmt="%s");
      trialNum = trialNum + 1;
    else:
      currRow = currRow + 1;
  return rowCount;


In [133]:
def readMessages(textFile, msgStatements):
  range1 = [i for i in range(0,7)]
  fileRead = pd.read_table(textFile, usecols = range1, sep='\t', delim_whitespace=True, dtype = {"TIMESTAMP":object, "RIGHT_EYE_X":object, "RIGHT_EYE_Y":object, "RIGHT_PUPIL_SIZE":object, "LEFT_EYE_X":object, "LEFT_EYE_Y":object, "LEFT_EYE_PUPIL":object});
  fileRead.columns = ["TIMESTAMP", "RIGHT_EYE_X", "RIGHT_EYE_Y", "RIGHT_PUPIL_SIZE", "LEFT_EYE_X", "LEFT_EYE_Y", "LEFT_EYE_PUPIL"];
  rowCount = fileRead.shape[0];
  colCount = 7;
  msg = "MSG";
  currRow = 0;
  while currRow < rowCount:
    startRow = 1;
    endRow = 1;
    if msg in str(fileRead.iloc[currRow, 0]):
      s = str(fileRead.iloc[currRow, 2]);
      res = s.translate(str.maketrans('','','1234567890'));
      msgStatements.append(res);
      #print(res)
      currRow = currRow + 1;
    else:
      currRow = currRow + 1;
  return msgStatements


In [169]:
#counts how many types of each trial there is, marks SSACC times, and drops unwanted messages
def countTypes(textFile, msgStatements, break_msg, msgCountPast):
  range1 = [i for i in range(0,7)]
  fileRead = pd.read_table(textFile, usecols = range1, sep='\t', delim_whitespace=True, dtype = {"TIMESTAMP":object, "RIGHT_EYE_X":object, "RIGHT_EYE_Y":object, "RIGHT_PUPIL_SIZE":object, "LEFT_EYE_X":object, "LEFT_EYE_Y":object, "LEFT_EYE_PUPIL":object});
  fileRead.columns = ["TIMESTAMP", "RIGHT_EYE_X", "RIGHT_EYE_Y", "RIGHT_PUPIL_SIZE", "LEFT_EYE_X", "LEFT_EYE_Y", "LEFT_EYE_PUPIL"];
  rowCount = fileRead.shape[0];
  colCount = 7;
  message = "MSG";
  # variables 
  msgCount = np.zeros(rowCount, dtype = int);
  msgState_count = msgCountPast;
  currRow = 0;
  while currRow < rowCount:
    startRow = 1;
    endRow = 1;
    s1 = str(fileRead.iloc[currRow, 2]);
    table1 = str.maketrans('', '', digits)
    res1 = s1.translate(table1);
    if res1 in break_msg:
      print(res1);
      fileRead.drop(currRow);
      np.savetxt(textFile, fileRead, delimiter=" ", fmt="%s");
      rowCount = rowCount - 1;
    if message in str(fileRead.iloc[currRow, 0]):
      s = str(fileRead.iloc[currRow, 2]);
      table = str.maketrans('', '', digits)
      res = s.translate(table);
      indexFound = msgStatements.index(res)
      if indexFound >= 0:
        msgState_count[indexFound] = msgState_count[indexFound] + 1;
        print("added count")
      currRow = currRow + 1;
    else:
      currRow = currRow + 1;

  return msgState_count;


In [11]:
#Run #1
rowCount = countTrials('/content/NLS_14_3.txt',)

In [15]:
# Run #2
APP_FOLDER = "/content/proceed1";
totalFiles = 0;
for base, dirs, files in os.walk(APP_FOLDER):
    for Files in files:
        totalFiles += 1
print(totalFiles)


71


In [ ]:
# Run #3
#what works: count MSG types (havent added count SSACC yet but this is relatively easy since its the same logic as count msg types)
#what doesn't work: dropping lines that have messages we don't want, "break_msg"
break_msg = {"EFIX", "ESACC", "SFIX", "EBLINK"};
sacc_msg = {"SSACC"};
msgStates = [];
patientNum = 14;
msgStatements = readMessages("/content/proceed1/NLS_"+str(patientNum)+"_1.txt", msgStates);
rowMsg = len(msgStatements);
msgPre = [0] * rowMsg;
msgCount = countTypes("/content/proceed1/NLS_"+str(patientNum)+"_1.txt", msgStatements, break_msg, msgPre)
print(msgCount)

In [ ]:
#msgStatements = ["Prosaccades", "Fixations_1"];
#STILL WORKING:
# need to find a way to keep msgStatements constantly updated with new messages without overlapping on old ones (current issue), 
# and also keep the index of messages stored to never change so that the numbers don't get messed up
for i in range(2,totalFiles):
 msgStatements = readMessages("/content/proceed1/NLS_"+str(patientNum)+"_"+str(i)+".txt", msgStatements);
 print(msgStatements)
 msgCount = countTypes("/content/proceed1/NLS_"+str(patientNum)+"_"+str(i)+".txt", msgStatements, break_msg, msgCount)

msgState_count = [[msgStatements], [msgCount]];
print(msgState_count)